In [1]:
import json

# Load the JSON file
with open('/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/models/scmamba/vocab.json', 'r') as f:
    vocab = json.load(f)

In [15]:
import dgl
dgl_graph = dgl.DGLGraph()

num_nodes = len(vocab)
dgl_graph.add_nodes(num_nodes)

import pandas as pd
edge_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/graph/tf_target_edgelist.csv")
edge_df.iloc[:, 0] = [vocab[g] for g in edge_df.iloc[:, 0]]
edge_df.iloc[:, 1] = [vocab[g] for g in edge_df.iloc[:, 1]]

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(
/tmp/ipykernel_869443/1601792552.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  edge_df.iloc[:, 0] = [vocab[g] for g in edge_df.iloc[:, 0]]
/tmp/ipykernel_869443/1601792552.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  edge_df.iloc[:, 1] = [vocab[g] for g in edge_df.iloc[:, 1]]


In [17]:
dgl_graph.add_edges(edge_df["source"].tolist(), edge_df["target"].tolist())

In [19]:
dgl.save_graphs(r'/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/graph/tf_target_cxg.dgl', dgl_graph)

In [1]:
import json

# Load the JSON file
with open('/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/models/scmamba/vocab.json', 'r') as f:
    vocab = json.load(f)

In [2]:
import pyarrow.feather as feather
import pandas as pd

df = feather.read_feather('/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings.feather')


motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0, low_memory=False)

df.index = df["motifs"].tolist()
df = df.drop("motifs", axis=1)

motif_dict = dict(zip(motif_df.index, motif_df["gene_name"]))
df = df.loc[df.index.isin(motif_dict.keys()), :]
df.index = [motif_dict[key] for key in df.index]

In [3]:
df = df.loc[df.index.isin(list(vocab.keys())), df.columns.isin(list(vocab.keys()))]

In [4]:
from tqdm import tqdm

def process_edges(df):
    
    pos_edges = []

    for i, row in tqdm(df.iterrows(), total=len(df) , desc="Processing TFs"):
        
        pos_edges.append(row.sort_values(ascending=True)[:50].index.tolist())
        
    pos_edges = pd.DataFrame(pos_edges, index=df.index)
    pos_edges = pos_edges.melt(ignore_index=False)
    pos_edges["Gene1"] = pos_edges.index.tolist()
    pos_edges["Gene2"] = pos_edges["value"].tolist()
    pos_edges = pos_edges.loc[:, ["Gene1", "Gene2"]]
    
    return pos_edges

In [5]:
edgeDF = process_edges(df)

Processing TFs: 100%|██████████| 16713/16713 [00:48<00:00, 343.59it/s]


In [6]:
import networkx as nx
g = nx.from_pandas_edgelist(edgeDF, source="Gene1", target="Gene2", create_using=nx.DiGraph)
print(len(g.edges))
g.remove_edges_from(nx.selfloop_edges(g))
print(len(g.edges))

339445
339312


In [7]:
def remove_cycle(graph):
    while True:
        try:
            cycle = nx.find_cycle(graph, orientation='original')
            print("Found cycle:", cycle)
            for u, v, _ in cycle:
                graph.remove_edge(u, v)
        except StopIteration:
            print("No more cycles detected.")
            break
    return graph

In [ ]:
g = remove_cycle(g)

Found cycle: [('PAX6', 'ZFHX3', 'forward'), ('ZFHX3', 'UNCX', 'forward'), ('UNCX', 'FOXP4', 'forward'), ('FOXP4', 'ZNF536', 'forward'), ('ZNF536', 'TCF7L2', 'forward'), ('TCF7L2', 'SP5', 'forward'), ('SP5', 'NKX2-2', 'forward'), ('NKX2-2', 'PAX6', 'forward')]
Found cycle: [('PAX6', 'FOXP4', 'forward'), ('FOXP4', 'FOXP1', 'forward'), ('FOXP1', 'MEIS1', 'forward'), ('MEIS1', 'HOXB3', 'forward'), ('HOXB3', 'PRDM13', 'forward'), ('PRDM13', 'PAX2', 'forward'), ('PAX2', 'SALL1', 'forward'), ('SALL1', 'ZEB1', 'forward'), ('ZEB1', 'ZFPM1', 'forward'), ('ZFPM1', 'SOX4', 'forward'), ('SOX4', 'NR2E1', 'forward'), ('NR2E1', 'PAX6', 'forward')]
Found cycle: [('FOXN1', 'NKX2-2', 'forward'), ('NKX2-2', 'HES5', 'forward'), ('HES5', 'TFDP1', 'forward'), ('TFDP1', 'EGR3', 'forward'), ('EGR3', 'FOXP2', 'forward'), ('FOXP2', 'FOXN1', 'forward')]
Found cycle: [('HOXA5', 'BARHL2', 'forward'), ('BARHL2', 'PAX7', 'forward'), ('PAX7', 'LBX1', 'forward'), ('LBX1', 'SP8', 'forward'), ('SP8', 'ING4', 'forward'), 